In [ ]:
#Read .fasta files from the entrada_genomas/ folder.

In [ ]:
import os
import glob

ruta_fasta = ".../entrada_genoma"
ruta_salida = ".../pharokka_out"

ficheros_fasta = glob.glob(os.path.join(ruta_fasta, "*.fasta"))
print(f"Se han encontrado {len(ficheros_fasta)} archivos .fasta")

genomas_por_anotar = []
for fasta in ficheros_fasta:
    nombre = os.path.splitext(os.path.basename(fasta))[0]
    carpeta_salida = os.path.join(ruta_salida, nombre)
    if not os.path.exists(carpeta_salida):
        genomas_por_anotar.append((nombre, fasta))

print(f"{len(genomas_por_anotar)} genomas necesitan anotación:")
for nombre, _ in genomas_por_anotar:
    print(f" - {nombre}")

In [ ]:
#Run Pharokka on the pending genomes.

In [ ]:
%%bash

source ~/.bashrc
conda activate pharokka-env

INPUT_DIR=".../entrada_genomas"
OUTPUT_DIR=".../pharokka_out"
DB_DIR=".../miniconda3/envs/pharokka-env/bin/databases"
PHAROKKA=".../miniconda3/envs/pharokka-env/bin/pharokka.py"

mkdir -p "$OUTPUT_DIR"

for file in "$INPUT_DIR"/*.fasta; do
    base=$(basename "$file" .fasta)
    
    if [ -d "$OUTPUT_DIR/$base" ]; then
        echo " $base ya anotado. Saltando..."
        continue
    fi

    echo "Anotando $base..."
    /home/alumno08/miniconda3/envs/pharokka-env/bin/python "$PHAROKKA" -i "$file" -o "$OUTPUT_DIR/$base" -d "$DB_DIR" -t 4 -f
done

echo "Anotación completa. Resultados guardados en $OUTPUT_DIR."

In [ ]:
#Construction of the binary matrix.

In [ ]:
import os
import pandas as pd
from tqdm import tqdm

PHAROKKA_OUT = ".../pharokka_out"
OUTFILE = ".../df_phrogs_binaria.tsv"

genomas_anotados = sorted([
    d for d in os.listdir(PHAROKKA_OUT)
    if os.path.isdir(os.path.join(PHAROKKA_OUT, d)) and not d.startswith(".")
])

phrog_dict = {}

for genoma_id in tqdm(genomas_anotados, desc="Procesando genomas anotados"):
    phrog_file = os.path.join(PHAROKKA_OUT, genoma_id, "pharokka_cds_final_merged_output.tsv")
    try:
        df_phrog = pd.read_csv(phrog_file, sep="\t")
        if "phrog" not in df_phrog.columns:
            raise ValueError("La columna 'phrog' no está en el archivo.")
        
        phrogs = df_phrog["phrog"].dropna().unique()
        phrogs = [str(p).strip().replace("phrog_", "") for p in phrogs if isinstance(p, str) and p != "No_PHROG"]
        phrog_dict[genoma_id] = phrogs

    except Exception as e:
        print(f"Error leyendo PHROGs en {genoma_id}: {e}")
        phrog_dict[genoma_id] = []

phrogs_unicos = sorted(set(
    phrog for lista in phrog_dict.values() for phrog in lista if isinstance(phrog, str)
))

df_bin = pd.DataFrame(0, index=genomas_anotados, columns=[f"PHROG_{p}" for p in phrogs_unicos])

for genoma_id, lista in phrog_dict.items():
    for phrog in lista:
        col = f"PHROG_{phrog}"
        if col in df_bin.columns:
            df_bin.at[genoma_id, col] = 1

df_bin.to_csv(OUTFILE, sep="\t")
print(f" Matriz binaria guardada correctamente en: {OUTFILE}")

In [ ]:
#Construction of the matrix with taxonomy and genomic information.

In [ ]:
import os
import pandas as pd
from tqdm import tqdm

PHAROKKA_OUT = ".../pharokka_out"
RESUMEN_PATH = ".../matriz_phrogs_tax_gc_simple.tsv"

genomas = sorted([
    d for d in os.listdir(PHAROKKA_OUT)
    if os.path.isdir(os.path.join(PHAROKKA_OUT, d)) and not d.startswith(".")
])

info_data = []

for genoma in tqdm(genomas, desc="Extrayendo información genómica"):
    resumen_path = os.path.join(PHAROKKA_OUT, genoma, "pharokka_length_gc_cds_density.tsv")
    phrog_path = os.path.join(PHAROKKA_OUT, genoma, "pharokka_cds_final_merged_output.tsv")
    
    try:
        resumen_df = pd.read_csv(resumen_path, sep="\t")
        length = resumen_df["length"].iloc[0]
        gc_perc = resumen_df["gc_perc"].iloc[0]
        
        if os.path.exists(phrog_path):
            genes_df = pd.read_csv(phrog_path, sep="\t")
            num_genes = genes_df.shape[0]
        else:
            num_genes = 0

        info_data.append({
            "genome_id": genoma,
            "genome_length": length,
            "gc_percent": gc_perc,
            "num_genes": num_genes
        })
    except Exception as e:
        print(f"Error en {genoma}: {e}")

df_info = pd.DataFrame(info_data).set_index("genome_id")
df_info.to_csv(RESUMEN_PATH, sep="\t")
print(f" Tabla taxonómica e información genómica guardada en {RESUMEN_PATH}")

In [ ]:
#Combination of the matrices.

In [ ]:
import pandas as pd
import os

# -----------------------------
# Rutas
# -----------------------------
ruta_binaria = ".../df_phrogs_binaria.tsv"
ruta_pharokka = ".../matriz_phrogs_tax_gc_simple.tsv"
ruta_phrogs = ".../phrogs_info"
ruta_salida = ".../matriz_phrogs_tax_gc_final.tsv"

niveles_tax = ["Kingdom", "Phylum", "Class", "Family", "Genus", "Species"]

df_bin = pd.read_csv(ruta_binaria, sep="\t", index_col=0)
df_pharokka = pd.read_csv(ruta_pharokka, sep="\t", index_col=0)

phrog_cols = [col for col in df_bin.columns if col.startswith("PHROG_")]

for col in phrog_cols:
    if col in df_pharokka.columns:
        df_bin[col] = df_bin[col] | df_pharokka[col]

phrog_cols_pharokka = [col for col in df_pharokka.columns if col.startswith("PHROG_")]
df_tax_gc = df_pharokka.drop(columns=phrog_cols_pharokka)

df_final = df_tax_gc.join(df_bin, how="outer").fillna(0)

phrogs_especificos = {nivel: pd.read_csv(f"{ruta_phrogs}/phrogs_especificos_por_{nivel.lower()}.tsv", sep="\t") for nivel in niveles_tax}
phrogs_caracteristicos = {nivel: pd.read_csv(f"{ruta_phrogs}/phrogs_caracteristicos_por_{nivel.lower()}.tsv", sep="\t") for nivel in niveles_tax}

phrog_cols_final = [col for col in df_final.columns if col.startswith("PHROG_")]
df_extra = pd.DataFrame(index=df_final.index)

for nivel in niveles_tax:
    df_esp = phrogs_especificos[nivel]
    df_carac = phrogs_caracteristicos[nivel]

    num_esp = []
    num_carac = []

    for idx, fila in df_final.iterrows():
        presentes = fila[phrog_cols_final][fila[phrog_cols_final] == 1].index.tolist()

        phrogs_esp_total = []
        phrogs_carac_total = []

        for col in df_esp.columns:
            phrogs_esp_total.extend(df_esp[col].dropna().tolist())
        for col in df_carac.columns:
            phrogs_carac_total.extend(df_carac[col].dropna().tolist())

        n_esp = 3 * len(set(presentes) & set(phrogs_esp_total))
        n_carac = len(set(presentes) & set(phrogs_carac_total))

        num_esp.append(n_esp)
        num_carac.append(n_carac)

    df_extra[f"especificos_{nivel.lower()}"] = num_esp
    df_extra[f"caracteristicos_{nivel.lower()}"] = num_carac

# -----------------------------
# Guardar matriz final
# -----------------------------
df_final = pd.concat([df_final, df_extra], axis=1)
df_final.to_csv(ruta_salida, sep="\t")

print(f" Matriz final combinada y corregida guardada en {ruta_salida}")

In [ ]:
#Application of predictive models and extraction of information with metrics and alerts.

In [ ]:
import pandas as pd
import pickle
import os

ruta_matriz = ".../matriz_phrogs_tax_gc_final.tsv"
ruta_modelos = ".../models"
ruta_salida = ".../predicciones_nuevos.tsv"

niveles_tax = ["Kingdom", "Phylum", "Class", "Family", "Genus", "Species"]

df = pd.read_csv(ruta_matriz, sep="\t", index_col=0)

resultados = pd.DataFrame(index=df.index)

for nivel in niveles_tax:
    
    print(f"\n Procesando nivel {nivel}...")

    # Cargar modelo
    with open(os.path.join(ruta_modelos, f"modelo_{nivel.lower()}.pkl"), "rb") as f:
        modelo = pickle.load(f)

    columnas_usadas = modelo.feature_names_in_.tolist()

    columnas_presentes = [col for col in columnas_usadas if col in df.columns]
    columnas_faltantes = [col for col in columnas_usadas if col not in df.columns]

    df_filtrado = df[columnas_presentes].copy()
    df_faltantes = pd.DataFrame(0, index=df.index, columns=columnas_faltantes)

    df_completo = pd.concat([df_filtrado, df_faltantes], axis=1)
    df_completo = df_completo[columnas_usadas]  # Orden correcto

    # Predicción
    predicciones = modelo.predict(df_completo)
    probs = modelo.predict_proba(df_completo)

    prob_predicha = []
    clases = modelo.classes_

    for pred, prob_vector in zip(predicciones, probs):
        idx_clase = list(clases).index(pred)
        prob_predicha.append(prob_vector[idx_clase])

    resultados[nivel] = predicciones
    resultados[f"Confianza_{nivel}"] = prob_predicha

resultados.to_csv(ruta_salida, sep="\t")
print(f"\n Predicciones guardadas en: {ruta_salida}")